In [2]:
import os
import sys

In [3]:
sys.path.append(os.getcwd() + "/hal_pm")

In [4]:
from hal_pm import *
import datetime

In [5]:
start_date = datetime.datetime(2018, 6, 1)

In [6]:
#Load Data
pmdata = Load_Data.load_data(48.5, 49, 9, 9.3, start_date, 1)

In [7]:
pmdata

,measurement_PM10,measurement_PM2.5,time,lat,lon,sensor_id,measurement_id
0,NaN,NaN,2018-06-01T00:00:01Z,48.798,9.070,11264,11264_2018-06-01T00:00:01Z
1,2.67,1.27,2018-06-01T00:00:01Z,48.949,9.098,122,122_2018-06-01T00:00:01Z
2,8.60,1.77,2018-06-01T00:00:01Z,48.765,9.147,4383,4383_2018-06-01T00:00:01Z
3,12.35,2.42,2018-06-01T00:00:01Z,48.779,9.034,181,181_2018-06-01T00:00:01Z
4,NaN,NaN,2018-06-01T00:00:01Z,48.779,9.034,182,182_2018-06-01T00:00:01Z
...,...,...,...,...,...,...,...
20129,3.90,1.33,2018-06-01T00:59:59Z,48.759,9.162,309,309_2018-06-01T00:59:59Z
20130,NaN,NaN,2018-06-01T00:59:59Z,48.536,9.274,202,202_2018-06-01T00:59:59Z
20131,NaN,NaN,2018-06-01T00:59:59Z,48.692,9.148,2493,2493_2018-06-01T00:59:59Z
20132,NaN,NaN,2018-06-01T00:59:59Z,48.799,9.224,5356,5356_2018-06-01T00:59:59Z
